# Multiple Linear Regression

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
import matplotlib.pyplot as plt

## Importing the DataSet


In [2]:
dataset = pd.read_csv('G:/Jupyter Sketch/Regression/Multiple Linear Regression/Data/wine quality-red.csv')
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
X_train = dataset.iloc[ : int((3/4)*(len(dataset)+1)),: -1].values
y_train = dataset.iloc[ : int((3/4)*(len(dataset)+1)), -1].values

In [4]:
X_test = dataset.iloc[len(X_train): , :-1].values
y_test = dataset.iloc[len(y_train): , -1].values

In [5]:
len(y_test)

399

In [6]:
cols = list(dataset.columns)

In [7]:
cols.remove('quality')

In [8]:
dataset = dataset[cols]

In [9]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


## Normalising the training and test dataset

In [10]:
def normalized(A):
    A = (A- np.min(A))/ (np.max(A) - np.min(A))
    return A

In [11]:
X_train = normalized(X_train)
y_train = normalized(y_train)
X_test = normalized(X_test)
y_test = normalized(y_test)

In [12]:
train_dataset = pd.DataFrame(X_train)

In [13]:
test_dataset = pd.DataFrame(X_test)

In [14]:
train_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.025606,0.002422,0.000000,0.006574,0.000263,0.038062,0.117647,0.003453,0.012145,0.001938,0.032526
1,0.026990,0.003045,0.000000,0.008997,0.000339,0.086505,0.231834,0.003449,0.011073,0.002353,0.033910
2,0.026990,0.002630,0.000138,0.007958,0.000318,0.051903,0.186851,0.003450,0.011280,0.002249,0.033910
3,0.038754,0.000969,0.001938,0.006574,0.000260,0.058824,0.207612,0.003453,0.010934,0.002007,0.033910
4,0.025606,0.002422,0.000000,0.006574,0.000263,0.038062,0.117647,0.003453,0.012145,0.001938,0.032526


In [15]:
X_train

array([[0.02560554, 0.00242215, 0.        , ..., 0.01214533, 0.00193772,
        0.03252595],
       [0.02698962, 0.00304498, 0.        , ..., 0.01107266, 0.00235294,
        0.03391003],
       [0.02698962, 0.00262976, 0.00013841, ..., 0.01128028, 0.00224913,
        0.03391003],
       ...,
       [0.0266436 , 0.00197232, 0.00072664, ..., 0.01093426, 0.00186851,
        0.03391003],
       [0.0266436 , 0.00089965, 0.00089965, ..., 0.01089965, 0.00273356,
        0.03771626],
       [0.02733564, 0.00200692, 0.00079585, ..., 0.01110727, 0.00200692,
        0.03287197]])

In [16]:
test_dataset.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.048125,0.003562,0.001312,0.009375,0.000431,0.02500,0.05625,0.006216,0.019750,0.003375,0.061250
1,0.049375,0.002125,0.002250,0.011875,0.000406,0.03125,0.06250,0.006214,0.020438,0.003375,0.070000
2,0.053750,0.002625,0.002437,0.011250,0.000425,0.03750,0.07500,0.006220,0.020938,0.004312,0.073125
3,0.061875,0.004625,0.001187,0.036250,0.000694,0.20625,0.47500,0.006242,0.019625,0.003438,0.058750
4,0.045000,0.002250,0.002875,0.013125,0.000463,0.15000,0.27500,0.006221,0.021250,0.005312,0.068750


## Normal Equation θ = Inv(X.XT).XT.Y

In [17]:
X0 = np.ones(shape=(len(X_train),1)) 

In [18]:
X_train = np.concatenate((X0,X_train), axis =1)

In [19]:
Xtrain_XT = np.dot(X_train.T, X_train) # Generally X.XT but np linear algebra is giving me an error of singular matrix

In [20]:
Xtrain_XT.shape

(12, 12)

In [21]:
y_train.shape

(1200,)

In [22]:
X_train.T.shape

(12, 1200)

In [23]:
Xtrain_XT_Inv = np.linalg.inv(Xtrain_XT)

In [24]:
Xtrain_XT_Inv_XT = np.dot(Xtrain_XT_Inv, X_train.T)

In [25]:
Xtrain_XT_Inv_XT.shape

(12, 1200)

In [26]:
thetas_Neq = np.dot(Xtrain_XT_Inv_XT,y_train)

In [27]:
thetas_Neq.shape

(12,)

In [28]:
thetas_Neq

array([ 7.92296708e+00,  2.70635193e+00, -6.17644133e+01, -1.55527126e+01,
        2.00203731e+00, -9.28035957e+01,  2.01557391e-01, -2.18139544e-01,
       -2.26175928e+03, -1.46716669e+01,  4.46143565e+01,  1.55746276e+01])

In [29]:
X_test.shape

(399, 11)

In [30]:
X0_test = np.ones(shape=(len(X_test),1))

In [31]:
X0_test.shape

(399, 1)

In [32]:
X_test.shape

(399, 11)

In [33]:
X_test = np.concatenate((X0_test,X_test), axis =1)

## Predction Function

In [34]:
def pred(X, theta):
    predct = np.dot(X, theta)
    return predct

In [35]:
y_pred_Neq = pred(X_test, thetas_Neq)


In [36]:
y_pred_Neq = normalized(y_pred_Neq)

In [37]:
y_pred_Neq.shape

(399,)

## Mean Square Error Function

In [38]:
def error(y_pred, y_test):
    err =  np.sum((y_pred - y_test)**2)/(2*len(y_test))
    return err


In [39]:
error_Neq = error(y_pred_Neq, y_test)
print('MSE = ',error_Neq)
y_pred_Neq

MSE =  0.014446819833745355


array([0.42885148, 0.68149471, 0.73170787, 0.25904511, 0.66326389,
       0.66326389, 0.66326389, 0.36874757, 0.66326389, 0.69246336,
       0.46285929, 0.3666301 , 0.46285929, 0.64346198, 0.58561019,
       0.70143947, 0.28414376, 0.83054936, 0.59070145, 0.70540255,
       0.704192  , 0.704192  , 0.27738265, 0.82541808, 0.65173964,
       0.28864753, 0.34822969, 0.38865046, 0.87257136, 0.34682992,
       0.82835459, 0.38083265, 0.34682992, 0.49700248, 0.67779521,
       0.67083358, 0.400828  , 0.67779521, 0.25450165, 0.62343326,
       0.27024473, 0.46556142, 0.78295997, 0.24285223, 0.54496832,
       0.46344305, 0.30799131, 0.46344305, 0.64108463, 0.52874935,
       0.52874935, 0.36087695, 0.29018882, 0.32466119, 0.49053703,
       0.46425185, 0.27899808, 0.49862622, 0.51884188, 0.51884188,
       0.21759253, 0.33513676, 0.43484658, 0.22648583, 0.75671079,
       0.45066537, 0.45066537, 0.79409266, 0.33386421, 0.96850056,
       1.        , 0.65739133, 0.57373229, 0.32244375, 0.54814

In [40]:
y_test

array([0.6, 0.8, 1. , 0.4, 0.8, 0.8, 0.8, 0.4, 0.8, 0.8, 0.6, 0.4, 0.6,
       0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.4,
       0.4, 0.4, 0.8, 0.4, 0.6, 0.4, 0.4, 0.2, 0.6, 0.2, 0.6, 0.6, 0.2,
       0.2, 0.4, 0.4, 0.6, 0.4, 0.6, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.4,
       0.4, 0.4, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.4, 0.2, 0.4, 0.2, 0.6,
       0.6, 0.6, 0.6, 0.6, 1. , 0.6, 0.6, 0.4, 0.4, 0.6, 0.6, 0.2, 0.6,
       0.6, 0.8, 0.6, 0.6, 0.6, 0.6, 0.4, 0.4, 0.6, 0.4, 0.4, 0.4, 0.4,
       0.6, 0.6, 0.2, 0.6, 0.4, 0.4, 0.6, 0.6, 0. , 0.6, 0.6, 0.6, 0.4,
       0.4, 0.4, 0.4, 0.2, 0.4, 0.4, 0.4, 0.6, 0.4, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.6, 0.6, 0.4, 0.6, 0.4, 0.8, 0.6, 0.6, 0.6, 0.6, 0.4, 0.6,
       0.6, 0.4, 0.6, 0.4, 0.4, 0.6, 0.4, 0.4, 0.4, 0.6, 0.6, 0.6, 0.6,
       0.6, 0.4, 0.6, 0.4, 0.4, 0.4, 0.4, 0.4, 0.6, 0.4, 0.4, 0.4, 0.4,
       0.4, 0.6, 0.4, 0.6, 0.4, 0.4, 0.6, 0.2, 0.6, 0.4, 0.4, 0.6, 0.6,
       0.2, 0.4, 0.6, 0.4, 0.4, 0. , 0.4, 0.4, 0.6, 0.6, 0.6, 0.

## Gradient Descent

In [41]:
def gradient_descent(X, y, thetas_gd, alpha, itrs):
    for i in range(itrs):
        for j in range(thetas_gd.shape[1]):
            thetas_gd[: j] = thetas_gd - (alpha/len(X))*(np.sum((np.dot(thetas_gd, X.T)-y)*X[:,j]))
    return thetas_gd

In [42]:
thetas_gd = np.random.rand(1, len(cols)+1) 

In [43]:
thetas_gd = gradient_descent(X_train, y_train, thetas_gd, 0.01, 100000)

In [44]:
thetas_gd

array([[0.3509742 , 0.84659039, 0.5244271 , 0.70367111, 1.12382264,
        0.67880577, 0.31174986, 0.37965524, 0.67529752, 0.32778848,
        0.57285552, 0.59673566]])

In [45]:
thetas_gd.shape

(1, 12)

In [46]:
X_test.shape

(399, 12)

In [47]:
y_pred_gd = pred(thetas_gd,X_test.T)

In [48]:
y_pred_gd

array([[0.48364113, 0.49716421, 0.5098365 , 0.74013792, 0.61374089,
        0.61374089, 0.61374089, 0.64253852, 0.61374089, 0.54167519,
        0.50750412, 0.61960607, 0.50750412, 0.51160365, 0.50584648,
        0.57840629, 0.66617706, 0.72853225, 0.53611107, 0.63268194,
        0.61929188, 0.61929188, 0.73613947, 0.55595537, 0.55387742,
        0.68615638, 0.6299726 , 0.53079372, 0.70110815, 0.6514653 ,
        0.63780194, 0.77424647, 0.6514653 , 0.58298155, 0.5311954 ,
        0.81172829, 0.50849777, 0.5311954 , 0.54304615, 0.54417843,
        0.66498213, 0.68178731, 0.59858527, 0.68721688, 1.04996848,
        0.56861286, 0.59789694, 0.56861286, 0.58963169, 0.59560232,
        0.59560232, 0.65351975, 0.49887633, 0.49645337, 0.58479607,
        0.61633559, 0.67907029, 0.66686641, 0.57171742, 0.57171742,
        0.63499748, 0.54741583, 0.69602108, 0.54833881, 0.64087615,
        0.57594475, 0.57594475, 0.55265507, 0.61522172, 0.72213959,
        0.64985327, 0.65878713, 0.60552223, 0.62

In [49]:
error_gd = error(y_pred_gd, y_test)
print('MSE_GD = ',error_gd)

MSE_GD =  0.021621935474684526
